### Tweet Semantic Search using EigenDB

Dataset: https://www.kaggle.com/datasets/kazanova/sentiment140/data (a subset of the first 1000 rows is used)

In [24]:
from eigen_py.index import EigenIndex
from eigen_py.data_types import Text

import pandas as pd
import os

from dotenv import load_dotenv
load_dotenv()

True

In [25]:
index = EigenIndex(
    url="http://localhost:8080",
    api_key="8685afc77dcf6965ce311d96d812d373",
    model_name="text-embedding-3-small",
    model_provider="openai",
    model_provider_api_key=os.getenv("OPENAI_TOKEN")
)
index

Authentication successful: Authenticated.


EigenIndex(url=http://localhost:8080)

In [37]:
df = pd.read_csv("./datasets/tweets/tweets.csv")
df

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
995,0,1468055262,Mon Apr 06 23:28:39 PDT 2009,NO_QUERY,LarryEisner,@dkoenigs thanks man. I'm so very grateful. ...
996,0,1468055266,Mon Apr 06 23:28:41 PDT 2009,NO_QUERY,ohmigosh_dusti,@t_wolfe i miss u too. i'm totally comin back...
997,0,1468055472,Mon Apr 06 23:28:43 PDT 2009,NO_QUERY,tiphaniebrooke,@sniffinglue ohhh. I love it. ps I'm sad we di...
998,0,1468055604,Mon Apr 06 23:28:45 PDT 2009,NO_QUERY,rinahannah,And somehow I still end up in this place


In [39]:
tweets = [Text(id=index, data=row['text'], metadata={"id": str(row['ids'])}) for index, row in df.iterrows()]
tweets[:5]

[Text(id=0 data=@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D, metadata={'id': '1467810369'}),
 Text(id=1 data=is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!, metadata={'id': '1467810672'}),
 Text(id=2 data=@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds, metadata={'id': '1467810917'}),
 Text(id=3 data=my whole body feels itchy and like its on fire , metadata={'id': '1467811184'}),
 Text(id=4 data=@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. , metadata={'id': '1467811193'})]

In [40]:
index.upsert_text(tweets)

In [41]:
results = index.search_text(text="I love eating amazing food", k=5)

# rank of 0 is the most similar to the query
for tweet in results.values():
    tweet_id = tweet['metadata']['id']
    tweet_data = df[df['ids'] == int(tweet_id)]
    print(f"Author: @{tweet_data['user'].values[0]}\nTweet: \"{tweet_data['text'].values[0]}\"\nRank: {tweet['rank']}\n")

Author: @BustaBusta
Tweet: "I know my life has been flipped upside down when I just thought in my head that some Ramen sounds good. "
Rank: 0

Author: @mitrepeak
Tweet: "http://twitpic.com/2y3cf - Filled with curry  the true indian in me is coming out"
Rank: 3

Author: @NikNaki08
Tweet: "i might break down and eat some buffalo wings tomorrow "
Rank: 2

Author: @Moonflowerchild
Tweet: "@Steve_Buscemi SHUT UP you...I am hungry! And in Britain, home of the most boring suckassy breakfast in the world!  I MISS DENNYS!!"
Rank: 1

Author: @tessthetraveler
Tweet: "@BATMANNN  i love chutney......"
Rank: 4



In [42]:
results = index.search_text(text="Travel", k=5)

# rank of 0 is the most similar to the query
for tweet in results.values():
    tweet_id = tweet['metadata']['id']
    tweet_data = df[df['ids'] == int(tweet_id)]
    print(f"Author: @{tweet_data['user'].values[0]}\nTweet: \"{tweet_data['text'].values[0]}\"\nRank: {tweet['rank']}\n")

Author: @JamieMphoto
Tweet: "@SaveTheStrib Dang! I would have done this if I knew it was there earlier.  Any other outings planned?"
Rank: 3

Author: @larabehnert
Tweet: "@trishzw megafast trip, you have time during day tom/weds? Or, you going out tonight? Want to see you bad. Tried to warn you, tech fail "
Rank: 0

Author: @wiwiwinDra
Tweet: "vacations makes me feel sick! "
Rank: 2

Author: @driveaway2008
Tweet: "@statravelAU just got ur newsletter, those fares really are unbelievable, shame I already booked and paid for mine "
Rank: 1

Author: @idrisjoel
Tweet: "is stucked in Paris and can't even travel into France (for work) "
Rank: 4

